## Криптографические протоколы

### Цель работы:

Реализовать протокол совместной выработки сеансового ключа Diffie-Hellman.
Требования к работе:
 * Разработка двух независимых модулей-участников протокола.
 * Реализовать атаку «человека посередине»
 * Отчет должен содержать описание протокола с указанием особенностей реализации, примеры работы программы.

### Ход работы:

**Протокол Диффи — Хеллмана (англ. Diffie–Hellman, DH)** — криптографический протокол, позволяющий двум и более сторонам получить общий секретный ключ, используя незащищенный от прослушивания канал связи. Полученный ключ используется для шифрования дальнейшего обмена с помощью алгоритмов симметричного шифрования.

#### Реализация на языке Python

In [1]:
class DiffieHellman:
    def __init__(self, g, p, private_key):
        self.g = g
        self.p = p
        self.private_key = private_key
        self.full_key = None

    def generate_public_key(self):
        public_key = (self.g ** self.private_key) % self.p
        return public_key

    def generate_full_key(self, public_key):
        full_key = (public_key ** self.private_key) % self.p
        self.full_key = full_key
        return full_key

    def encrypt_message(self, message):
        encrypted_message = ""
        for c in message:
            encrypted_message += chr(ord(c) + self.full_key)
        return encrypted_message

    def decrypt_message(self, encrypted_message):
        decrypted_message = ""
        for c in encrypted_message:
            decrypted_message += chr(ord(c) - self.full_key)
        return decrypted_message
    
    def __str__(self):
        return f'g = {self.g}\np = {self.p}\nprivate_key = {self.private_key}\nfull_key = {self.full_key}'

В классе **DiffieHellman** представлены следующие методы:

`__init__(self, g, p, private_key)` - конструктор для создания абонента;

`generate_public_key(self)` - метод генерации публичного ключа;

`generate_full_key(self, public_key)` - метод генерации полного общего ключа, принимающий в качестве параметра публичный ключ;

`encrypt_message(self, message)` - метод шифрования сообщения на основе шифра простой замены (сдвига);

`decrypt_message(self, encrypted_message)` - метод дешифрования сообщения на основе шифра простой замены (сдвига);

`__str__(self)` - метод для вывода полной информации об абоненте

Зададим исходные параметры

In [2]:
message = "Secret message"
g = 2
p = 211
a = 23
b = 31

In [3]:
alice = DiffieHellman(g, p, a)
bob = DiffieHellman(g, p, b)

Вычисление публичных ключей

$$ga = g^a mod p$$

In [4]:
ga = alice.generate_public_key()

In [5]:
ga

92

$$gb = g^b mod p$$

In [6]:
gb = bob.generate_public_key()

In [7]:
gb

131

Обмен публичными ключами и генерация общего секретного ключа

$$K_a = gb^a mod p = g^{ab} mod p$$

In [8]:
alice_full_key = alice.generate_full_key(gb)

In [9]:
alice_full_key

133

$$K_b = ga^b mod p = g^{ab} mod p$$

In [10]:
bob_full_key = bob.generate_full_key(ga)

In [11]:
bob_full_key

133

Таким образом можно заметить, что ключи Алисы и Боба одинаковы

$$K = g^{ab} mod p$$

Обмен сообщениями с помощью общего секретного ключа. Для демонстрации используется шифр простой замены.

In [12]:
encrypted_message = alice.encrypt_message(message)

In [13]:
encrypted_message

'Øêè÷êù¥òêøøæìê'

In [14]:
decrypted_message = bob.decrypt_message(encrypted_message)

In [15]:
decrypted_message

'Secret message'

### Атака "человек посередине"

**Атака «человек посередине» (англ. Man in the middle (MITM))** — вид атаки в криптографии и компьютерной безопасности, когда злоумышленник тайно ретранслирует и при необходимости изменяет связь между двумя сторонами, которые считают, что они непосредственно общаются друг с другом. Является методом компрометации канала связи, при котором взломщик, подключившись к каналу между контрагентами, осуществляет вмешательство в протокол передачи, удаляя или искажая информацию.

Ни Алиса, ни Боб не могут достоверно определить, кем является их собеседник, поэтому вполне возможно представить случай, при котором Боб и Алиса установили связь с Евой, которая Алисе выдает себя за Боба, а Бобу представляется Алисой.

У Евы есть свои секретные ключи для общения с Бобом (eve_a) и Алисой (eve_b)

In [16]:
message2 = "New secret message"
eve_a = 13
eve_b = 27

Ева генерирует публичные ключи

In [17]:
eve_ga = (g ** eve_a) % p
eve_gb = (g ** eve_b) % p

Передаёт их Алисе и Бобу, которые генерируют общие секретные ключи. Но в данном случае они не будут одинаковыми, так как Ева имеет один общий ключ с Алисой (которая считает, что это Боб) и один общий ключ с Бобом (который считает, что это Алиса). А, следовательно, она может получать от Алисы любое сообщение для Боба, расшифровать его ключом, прочитать, изменить, зашифровать ключом и передать Бобу. Таким образом, подлог может оставаться незамеченным очень долгое время.

In [18]:
eve_alice_full_key = alice.generate_full_key(eve_gb)

In [19]:
eve_alice_full_key

68

In [20]:
eve_bob_full_key = bob.generate_full_key(eve_ga)

In [21]:
eve_bob_full_key

175

In [22]:
print(alice)

g = 2
p = 211
private_key = 23
full_key = 68


In [23]:
print(bob)

g = 2
p = 211
private_key = 31
full_key = 175


Алиса шифрует сообщение с помощью своего полного секретного ключа

In [24]:
encrypted_message2 = alice.encrypt_message(message2)

In [25]:
encrypted_message2

'\x92©»d·©§¶©¸d±©··¥«©'

Ева дешифрует сообщение Алисы и читает его

In [26]:
eve_decrypted_message = ""
for c in encrypted_message2:
    eve_decrypted_message += chr(ord(c) - eve_alice_full_key)

In [27]:
eve_decrypted_message

'New secret message'

Ева изменяет сообщение Алисы и шифрует его общим ключом с Бобом

In [28]:
eve_message = "Compromised secret message"
eve_encrypted_message = ""
for c in eve_message:
    eve_encrypted_message += chr(ord(c) + eve_bob_full_key)

In [29]:
eve_encrypted_message

'òĞĜğġĞĜĘĢĔēÏĢĔĒġĔģÏĜĔĢĢĐĖĔ'

Боб дешифрует сообщение и получает совсем не то, что отправляла ему Алиса, а сообщение, отправленное Евой.

In [30]:
decrypted_message2 = bob.decrypt_message(eve_encrypted_message)

In [31]:
decrypted_message2

'Compromised secret message'

### Выводы

Реализован протокол совместной выработки сеансового ключа Diffie-Hellman на языке Python. Также реализована атака "человек посередине", вследствие чего установлено, что протокол Diffie-Hellman уязвим к данной атаке. Поэтому для его использования необходимо использовать дополнительные методы защиты (цифровая подпись, хэш-функция и т.д.).